# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
import tensorflow as tf

In [1]:
import keras

Using TensorFlow backend.


In [0]:
(X_train,Y_train),(X_test,Y_test)=keras.datasets.cifar10.load_data()

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
from keras import utils

In [90]:
X_train.shape

(50000, 32, 32, 3)

In [0]:
Y_train=utils.to_categorical(Y_train,num_classes=10)

In [0]:
Y_test=utils.to_categorical(Y_test,num_classes=10)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D,MaxPooling2D

In [103]:
num_classes=10
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),strides=(1,1),activation='relu',input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(GlobalAveragePooling2D())
model.add(Dense(1000,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 13, 13, 32)        9248      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 4, 4, 64)          18496     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 2, 2, 64)          0         
_________________________________________________________________
global_average_pooling2d_4 ( (None, 64)                0         
__________

In [0]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [106]:
batch_size=100
epochs=10
model.fit(X_train,Y_train,batch_size=batch_size, epochs=epochs, validation_data=(X_test,Y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 66s 1ms/step - loss: 0.1773 - acc: 0.9303 - val_loss: 0.1862 - val_acc: 0.9283
Epoch 2/10
50000/50000 [==============================] - 68s 1ms/step - loss: 0.1664 - acc: 0.9347 - val_loss: 0.1687 - val_acc: 0.9337
Epoch 3/10
50000/50000 [==============================] - 64s 1ms/step - loss: 0.1564 - acc: 0.9389 - val_loss: 0.1606 - val_acc: 0.9378
Epoch 4/10
50000/50000 [==============================] - 65s 1ms/step - loss: 0.1478 - acc: 0.9423 - val_loss: 0.1662 - val_acc: 0.9356
Epoch 5/10
50000/50000 [==============================] - 67s 1ms/step - loss: 0.1402 - acc: 0.9448 - val_loss: 0.1757 - val_acc: 0.9322
Epoch 6/10
50000/50000 [==============================] - 65s 1ms/step - loss: 0.1353 - acc: 0.9468 - val_loss: 0.1692 - val_acc: 0.9354
Epoch 7/10
50000/50000 [==============================] - 68s 1ms/step - loss: 0.1307 - acc: 0.9490 - val_loss: 0.1537 - val_acc

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [107]:
model.layers

In [0]:
for layer in model.layers[:7]:
    layer.trainable = False

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [111]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

I0623 07:21:46.124352 140305953986432 twenty_newsgroups.py:247] Downloading 20news dataset. This may take a few minutes.
I0623 07:21:46.130924 140305953986432 twenty_newsgroups.py:80] Downloading dataset from https://ndownloader.figshare.com/files/5975967 (14 MB)


### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [113]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [114]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [115]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score